# Projet 8 : Traitez les images pour le système embarqué d’une voiture autonome

* [1. Contexte](#partie1)
* [2. Préparation de l'environnement](#partie2)
    * [2.1 Installation des modules](#partie2.1)
    * [2.2 Librairies](#partie2.2)
    * [2.3 Fonctions](#partie2.3)
* [3. Prétraitement des données](#partie3)
    * [3.1 Compléter le filtrage des groupes](#partie3.1)
    * [3.2 Data augmentation](#partie3.2)
    * [3.3 Data generator](#partie3.3)
* [4. Recherche de la meilleure fonction de perte](#partie4)
    * [4.1 Charger le pipeline d'augmentation et initialiser les générateurs](#partie4.1)
    * [4.2 Fonctions de perte](#partie4.2)
    * [4.3 Modèle Unet Mini](#partie4.3)
      * [4.3.1 Conception du modèle](#partie4.3.1)
      * [4.3.2 Compilation et entrainement](#partie4.3.2)
* [5. Conclusion](#partie5)

## <font color='red'>1. Contexte</font><a class="anchor" id="partie1"></a>

Ce notebook a pour objectif de tester 5 fonctions de perte adaptées à la segmentation d'images, en utilisant un modèle U-Net Mini.

Les fonctions testées sont :
- Dice Loss
- Total Loss
- IoU Loss
- Tversky Loss
- Focal Tversky Loss

## <font color='red'>2. Préparation de l'environnement</font><a class="anchor" id="partie2"></a>

##### <font color='blue'>2.1 Installation des modules</font><a class="anchor" id="partie2.1"></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.9 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow==2.10 keras==2.10 segmentation-models

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 50.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-

In [ ]:
!pip install albumentations

In [ ]:
!pip install pyngrok

In [ ]:
import subprocess
from pyngrok import ngrok

# Démarrer MLFlow en arrière-plan
mlflow_server = subprocess.Popen(["mlflow", "ui", "--port", "5000"])

# Ajouter mon authtoken Ngrok
!ngrok config add-authtoken 2o7fRSSTsKkRHY1jl4VoX9qS7AR_5TCYPvixQ8rv7g5PqJp8t

# Créer un tunnel pour accéder à MLFlow UI
public_url = ngrok.connect(5000)
print("MLFlow Tracking UI:", public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
MLFlow Tracking UI: NgrokTunnel: "https://7f2a-34-143-131-9.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
import mlflow
mlflow.set_experiment("P8 - Image Segmentation - recherche")

2025/01/13 11:23:33 INFO mlflow.tracking.fluent: Experiment with name 'P8 - Image Segmentation - recherche' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/179292789104252946', creation_time=1736767413939, experiment_id='179292789104252946', last_update_time=1736767413939, lifecycle_stage='active', name='P8 - Image Segmentation - recherche', tags={}>

##### <font color='blue'>2.2 Librairies</font><a class="anchor" id="partie2.2"></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image
import segmentation_models as sm
import albumentations as A
import mlflow
import tensorflow as tf
import time
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, Activation, MaxPooling2D, UpSampling2D, Concatenate, GlobalAveragePooling2D, Reshape, Lambda, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import  jaccard_score
from segmentation_models.metrics import IOUScore, FScore
from segmentation_models.losses import DiceLoss
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import MeanIoU
import cv2


Segmentation Models: using `keras` framework.


##### <font color='blue'>2.3 Fonctions</font><a class="anchor" id="partie2.3"></a>

In [ ]:
def load_file_paths(images_path, masks_path, data_cat="train"):
    """
    Charge les chemins des images et des masques pour un type de données spécifique.

    Args:
        images_path (str): Chemin du dossier des images.
        masks_path (str): Chemin du dossier des masques.
        data_cat (str): Type de données ("train", "val", "test").

    Returns:
        list, list: Listes des chemins des images et des masques.
    """
    images_dir = os.path.join(images_path, data_cat)
    masks_dir = os.path.join(masks_path, data_cat)

    # Vérification des répertoires
    if not os.path.exists(images_dir):
        raise FileNotFoundError(f"Le répertoire {images_dir} n'existe pas.")
    if not os.path.exists(masks_dir):
        raise FileNotFoundError(f"Le répertoire {masks_dir} n'existe pas.")

    cities = os.listdir(images_dir)
    image_files, mask_files = [], []

    for city in cities:
        city_img_dir = os.path.join(images_dir, city)
        city_mask_dir = os.path.join(masks_dir, city)

        # Vérifier si le sous-dossier existe dans "masks_path"
        if not os.path.exists(city_mask_dir):
            print(f"Attention : Le dossier des masques pour la ville '{city}' est manquant.")
            continue

        # Charger les fichiers
        city_images = [os.path.join(city_img_dir, f) for f in os.listdir(city_img_dir) if f.endswith("_leftImg8bit.png")]
        city_masks = [os.path.join(city_mask_dir, f) for f in os.listdir(city_mask_dir) if f.endswith("_gtFine_labelIds.png")]

        if len(city_images) != len(city_masks):
            print(f"Attention : Incohérence dans le nombre de fichiers pour la ville '{city}'.")

        image_files.extend(sorted(city_images))
        mask_files.extend(sorted(city_masks))

    return image_files, mask_files

In [ ]:
def show_image_and_mask(image_path, mask_path, save_path=None):
    """
    Affiche une image et son masque côte à côte.
    Si un chemin est fourni, enregistre la figure au format PNG.

    Args:
        image_path (str): Chemin de l'image RGB.
        mask_path (str): Chemin du masque associé.
        save_path (str, optional): Chemin pour sauvegarder l'image.
    """
    img = Image.open(image_path)
    mask = Image.open(mask_path)

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    ax[0].imshow(img)
    ax[0].set_title("Image RGB")
    ax[0].axis("off")

    ax[1].imshow(mask)
    ax[1].set_title("Masque (color)")
    ax[1].axis("off")

    # Enregistrer l’image si un chemin est fourni
    if save_path:
        fig.savefig(save_path, bbox_inches="tight")

    plt.show()

In [ ]:
def filter_groups_in_mask(mask_path, class_to_group):
    mask = np.array(Image.open(mask_path))
    group_mask = np.zeros_like(mask)
    for cls, grp in class_to_group.items():
        group_mask[mask == cls] = grp
    return group_mask

def apply_cityscapes_palette(group_mask):
    cityscapes_palette = [
        (128, 64, 128),  # road (flat)
        (244, 35, 232),  # sidewalk (flat)
        (70, 70, 70),    # building (construction)
        (102, 102, 156), # wall (construction)
        (190, 153, 153), # fence (construction)
        (153, 153, 153), # pole (object)
        (250, 170, 30),  # traffic light (object)
        (220, 220, 0),   # traffic sign (object)
        (107, 142, 35),  # vegetation (nature)
        (152, 251, 152), # terrain (nature)
        (70, 130, 180),  # sky (sky)
        (220, 20, 60),   # person (human)
        (255, 0, 0),     # rider (human)
        (0, 0, 142),     # car (vehicle)
        (0, 0, 70),      # truck (vehicle)
        (0, 60, 100),    # bus (vehicle)
        (0, 80, 100),    # on rails (vehicle)
        (0, 0, 230),     # motorcycle (vehicle)
        (119, 11, 32),   # bicycle (vehicle)
        (0, 0, 0)        # void
    ] + [(0, 0, 0)] * (256 - 20)

    pil_mask = Image.fromarray(group_mask.astype('uint8'))
    flat_palette = [value for color in cityscapes_palette for value in color]
    pil_mask.putpalette(flat_palette)
    return pil_mask

In [ ]:
import mlflow
import os

# **Démarrer un run MLFlow**
with mlflow.start_run(run_name="EDA_Analyse_Structure"):

    # Définir les chemins vers les images et les masques
    images_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit"
    masks_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_gtFine_trainvaltest/gtFine"

    # Vérification des dossiers
    images_dirs = os.listdir(images_path)
    masks_dirs = os.listdir(masks_path)

    print("Contenu des dossiers :")
    print("Images :", images_dirs)
    print("Masques :", masks_dirs)

    # **Enregistrement des informations dans MLFlow**
    mlflow.log_param("Nombre de dossiers images", len(images_dirs))
    mlflow.log_param("Nombre de dossiers masques", len(masks_dirs))

    # **Chargement des chemins des fichiers (sans les intégrer à MLFlow maintenant)**
    train_images, train_masks = load_file_paths(images_path, masks_path, "train")
    val_images, val_masks = load_file_paths(images_path, masks_path, "val")
    test_images, test_masks = load_file_paths(images_path, masks_path, "test")

    # **Enregistrement des métriques dans MLFlow**
    mlflow.log_metric("nb_images_train", len(train_images))
    mlflow.log_metric("nb_masks_train", len(train_masks))
    mlflow.log_metric("nb_images_val", len(val_images))
    mlflow.log_metric("nb_masks_val", len(val_masks))
    mlflow.log_metric("nb_images_test", len(test_images))
    mlflow.log_metric("nb_masks_test", len(test_masks))

    print("Analyse de la structure des fichiers terminée et enregistrée dans MLFlow.")


Contenu des dossiers :
Images : ['train', 'val', 'test']
Masques : ['test', 'val', 'train']
Analyse de la structure des fichiers terminée et enregistrée dans MLFlow.


## <font color='red'>3. Prétraitement des données</font><a class="anchor" id="partie3"></a>

##### <font color='blue'>3.1 Compléter le Filtrage des Groupes</font><a class="anchor" id="partie3.1"></a>

In [ ]:
with mlflow.start_run(run_name="EDA_Filtrage_Groupes"):
    # Définition des classes et groupes
    class_to_group = {
        -1: 0, 0: 0, 1: 0, 2: 0,  # void
        7: 1, 8: 1, 9: 1, 10: 1,  # flat
        11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2,  # construction
        17: 3, 18: 3, 19: 3, 20: 3,  # object
        21: 4, 22: 4,  # nature
        23: 5,  # sky
        24: 6, 25: 6,  # human
        26: 7, 27: 7, 28: 7, 29: 7, 30: 7, 31: 7, 32: 7, 33: 7  # vehicle
    }

    example_mask_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_gtFine_trainvaltest/gtFine/train/aachen/aachen_000000_000019_gtFine_labelIds.png"

    # Application du filtrage des groupes
    filtered_group_mask = filter_groups_in_mask(example_mask_path, class_to_group)
    colored_mask = apply_cityscapes_palette(filtered_group_mask)

    # Loguer le nombre de pixels par groupe
    unique, counts = np.unique(filtered_group_mask, return_counts=True)
    group_distribution = dict(zip(unique, counts))

    for group, count in group_distribution.items():
        mlflow.log_metric(f"group_{group}_pixels", count)

    # Sauvegarder l'image filtrée et colorisée dans MLFlow
    mask_output_path = "filtered_mask.png"
    colored_mask.save(mask_output_path)
    mlflow.log_artifact(mask_output_path)

    print("Filtrage des groupes et enregistrement des résultats terminés dans MLFlow.")

##### <font color='blue'>3.2 Data augmentation</font><a class="anchor" id="partie3.2"></a>

In [ ]:
def get_augmentation_pipeline():
    """
    Crée une pipeline d'augmentations dynamiques pour les images et les masques.
    Returns:
        albumentations.Compose: Pipeline d'augmentations.
    """
    augmentation = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.GaussNoise(p=0.1),
        A.Blur(blur_limit=(3, 7), p=0.1),
        A.Rotate(limit=10, p=0.3),
        A.Resize(height=256, width=256)
    ])

    # Loguer la configuration de l'augmentation dans MLFlow
    mlflow.log_param("augmentation_pipeline", str(augmentation))

    return augmentation

##### <font color='blue'>3.3 Data generator</font><a class="anchor" id="partie3.3"></a>

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    """
    Générateur de données avec augmentation dynamique pour l'entraînement et la validation.
    """

    def __init__(self, image_paths, mask_paths, batch_size, img_size=(256, 256), augmentation=None, shuffle=True, class_to_group=None, num_classes=8):
        """
        Initialisation du générateur de données.
        """
        assert len(image_paths) == len(mask_paths), "Les listes d'images et de masques doivent avoir la même longueur."
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.img_size = img_size
        self.augmentation = augmentation
        self.shuffle = shuffle
        self.class_to_group = class_to_group
        self.num_classes = num_classes
        self.indexes = np.arange(len(self.image_paths))
        self.on_epoch_end()

        # Loguer les paramètres du DataGenerator dans MLFlow
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("image_size", img_size)
        mlflow.log_param("num_classes", num_classes)
        mlflow.log_param("shuffle", shuffle)

    def __len__(self):
        """Nombre de lots par époque."""
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        """Génère un lot de données."""
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        image_paths_temp = [self.image_paths[k] for k in indexes]
        mask_paths_temp = [self.mask_paths[k] for k in indexes]
        X, y = self.__data_generation(image_paths_temp, mask_paths_temp)
        return X, y

    def on_epoch_end(self):
        """Mélange les données après chaque époque."""
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, image_paths_temp, mask_paths_temp):
        """Prépare les lots."""
        X = np.empty((self.batch_size, *self.img_size, 3), dtype=np.float32)
        y = np.empty((self.batch_size, *self.img_size, self.num_classes), dtype=np.float32)

        i = 0
        while i < self.batch_size:
            idx = np.random.randint(len(image_paths_temp))
            image_path = image_paths_temp[idx]
            mask_path = mask_paths_temp[idx]

            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (self.img_size[1], self.img_size[0]))

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (self.img_size[1], self.img_size[0]), interpolation=cv2.INTER_NEAREST)  # ✅ Correction

            unique_classes = np.unique(mask)
            if len(unique_classes) < 3:
                continue

            if self.augmentation:
                augmented = self.augmentation(image=image, mask=mask)
                image, mask = augmented['image'], augmented['mask']

            X[i] = image / 255.0
            y[i] = self._remap_and_one_hot_encode(mask)
            i += 1

        return X, y

    def _remap_and_one_hot_encode(self, mask):
        """Remappe les classes de masque et effectue un encodage one-hot."""
        remapped_mask = np.zeros_like(mask, dtype=np.int32)

        if self.class_to_group:
            for cls, grp in self.class_to_group.items():
                remapped_mask[mask == cls] = grp
        else:
            remapped_mask = mask

        one_hot_mask = tf.keras.utils.to_categorical(remapped_mask, num_classes=self.num_classes)
        return one_hot_mask.astype('float32')

    def compute_class_weights(self):
        """Calcule les poids pour chaque classe et les logue dans MLFlow."""
        pixel_counts = np.zeros(self.num_classes, dtype=np.int64)

        for mask_path in self.mask_paths:
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, self.img_size[::-1], interpolation=cv2.INTER_NEAREST)

            for cls, grp in self.class_to_group.items():
                pixel_counts[grp] += np.sum(mask == cls)

        total_pixels = np.sum(pixel_counts)
        class_weights = total_pixels / (self.num_classes * pixel_counts)
        normalized_weights = class_weights / np.sum(class_weights)

        # Loguer les poids des classes dans MLFlow
        for i, weight in enumerate(normalized_weights):
            mlflow.log_param(f"class_weight_{i}", float(weight))

        return normalized_weights

## <font color='red'>4. Recherche de la meilleure fonction de perte</font><a class="anchor" id="partie4"></a>

##### <font color='blue'>4.1 Charger le pipeline d'augmentation et initialiser les générateurs</font><a class="anchor" id="partie4.1"></a>

In [ ]:
with mlflow.start_run(run_name="Preprocessing"):
    # Charger les pipelines d'augmentation et les loguer dans MLFlow
    augmentation_pipeline = get_augmentation_pipeline()

    # Initialiser les générateurs
    train_gen = DataGenerator(
        train_images,
        train_masks,
        batch_size=16,
        img_size=(256, 256),
        augmentation=None,
        class_to_group=class_to_group,
        num_classes=8
    )

    val_gen = DataGenerator(
        val_images,
        val_masks,
        batch_size=16,
        img_size=(256, 256),
        augmentation=None,
        class_to_group=class_to_group,
        num_classes=8
    )

    # Loguer la taille des images dans MLFlow
    mlflow.log_param("generator_img_size", train_gen.img_size)

    # Calculer et loguer les poids des classes
    class_weights = train_gen.compute_class_weights()
    mlflow.log_dict({"class_weights": class_weights.tolist()}, "class_weights.json")

##### <font color='blue'> 4.2 Fonctions de perte</font><a class="anchor" id="partie4.2"></a>

In [ ]:
def dice_coeff(y_true, y_pred):
    """
    Calcule le coefficient de Dice, une mesure de similarité pour la segmentation.

    Args:
        y_true (Tensor): Masques vrais (ground truth).
        y_pred (Tensor): Masques prédits.

    Returns:
        float: Coefficient de Dice.
    """
    smooth = 1.0
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

In [ ]:
from tensorflow.keras.losses import binary_crossentropy

def total_loss(y_true, y_pred):
    """
    Combine binary_crossentropy et dice_loss pour améliorer les performances globales.

    Args:
        y_true (Tensor): Masques réels.
        y_pred (Tensor): Masques prédits.

    Returns:
        Tensor: Valeur de la perte combinée.
    """
    loss = binary_crossentropy(y_true, y_pred) + (3 * dice_loss(y_true, y_pred))
    return loss


In [ ]:
from tensorflow.keras import backend as K

def iou_loss(y_true, y_pred):
    """
    Calcule la perte basée sur l'IoU (Jaccard Loss).

    Args:
        y_true (Tensor): Masques vrais (ground truth).
        y_pred (Tensor): Masques prédits.

    Returns:
        Tensor: Valeur de la perte IoU.
    """
    smooth = 1e-6  # Pour éviter la division par zéro
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    return 1 - (intersection + smooth) / (union + smooth)


In [ ]:
def tversky_loss(y_true, y_pred, alpha=0.3, beta=0.7):
    """
    Calcule la Tversky Loss pour pondérer différemment les faux positifs et négatifs.

    Args:
        y_true (Tensor): Masques vrais.
        y_pred (Tensor): Masques prédits.
        alpha (float): Poids des faux positifs.
        beta (float): Poids des faux négatifs.

    Returns:
        Tensor: Valeur de la perte Tversky.
    """
    smooth = 1e-6
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)

    TP = K.sum(y_true_f * y_pred_f)
    FP = K.sum((1 - y_true_f) * y_pred_f)
    FN = K.sum(y_true_f * (1 - y_pred_f))

    return 1 - ((TP + smooth) / (TP + alpha * FP + beta * FN + smooth))


In [ ]:
def focal_tversky_loss(y_true, y_pred, alpha=0.3, beta=0.7, gamma=1.5):
    """
    Calcule la Focal Tversky Loss pour accorder plus d'importance aux pixels mal classifiés.

    Args:
        y_true (Tensor): Masques vrais.
        y_pred (Tensor): Masques prédits.
        alpha (float): Poids des faux positifs.
        beta (float): Poids des faux négatifs.
        gamma (float): Facteur de focalisation.

    Returns:
        Tensor: Valeur de la perte Focal Tversky.
    """
    return K.pow(tversky_loss(y_true, y_pred, alpha, beta), gamma)


##### <font color='blue'>4.3 Modèle U-Net Mini</font><a class="anchor" id="partie4.3"></a>

###### <font color='green'>4.3.1 Conception du Modèle</font><a class="anchor" id="partie4.3.1"></a>

In [ ]:
def unet_mini(input_size=(256, 256, 3), n_classes=32):
    """
    Implémente un modèle U-Net simplifié avec des convolutions dilatées.

    Args:
        input_size (tuple): Dimensions des images d'entrée (par défaut 256x256x3).
        n_classes (int): Nombre de classes pour la segmentation (par défaut 8).

    Returns:
        Model: Modèle U-Net Mini.
    """
    inputs = Input(input_size)

    # Encodeur
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Goulot d'étranglement avec convolution dilatée
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same', dilation_rate=2)(p2)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same', dilation_rate=4)(c3)
    c3 = BatchNormalization()(c3)

    # Décodeur
    u1 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(c3)
    u1 = concatenate([u1, c2])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    c4 = BatchNormalization()(c4)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    c4 = BatchNormalization()(c4)

    u2 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(c4)
    u2 = concatenate([u2, c1])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    c5 = BatchNormalization()(c5)
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(c5)
    c5 = BatchNormalization()(c5)

    # Couche de sortie
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c5)

    return Model(inputs, outputs)

with mlflow.start_run(run_name="Unet Mini - Conception") as run:
    start_time = time.time()  # Début chrono

    # Instancier le modèle
    unet_model = unet_mini(input_size=(256, 256, 3), n_classes=8)

    # Afficher le résumé du modèle
    model_summary = []
    unet_model.summary(print_fn=lambda x: model_summary.append(x))
    model_summary = "\n".join(model_summary)

    # Enregistrer le résumé du modèle dans MLFlow
    mlflow.log_text(model_summary, "unet_mini_summary.txt")

    # Temps de conception du modèle
    elapsed_time = time.time() - start_time
    mlflow.log_metric("conception_time", elapsed_time)

print(f"Modèle U-Net Mini conçu et loggé dans MLFlow en {elapsed_time:.2f} secondes.")

Modèle U-Net Mini conçu et loggé dans MLFlow en 0.80 secondes.


###### <font color='green'>4.3.2 Compilation et entrainement</font><a class="anchor" id="partie4.3.2"></a>

In [ ]:
# Liste des fonctions de perte et leur nom
loss_functions = {
    "Dice Loss": DiceLoss(),
    "Total Loss": total_loss,
    "IoU Loss": iou_loss,
    "Tversky Loss": tversky_loss,
    "Focal Tversky Loss": focal_tversky_loss
}

dice_loss = DiceLoss()

# Stocker les résultats
results = []

# Boucle d'entraînement avec chaque fonction de perte
for loss_name, loss_fn in loss_functions.items():
    print(f"Entraînement avec {loss_name}...")
    start_time = time.time()

    # Instancier le modèle U-Net Mini
    unet_model = unet_mini(input_size=(256, 256, 3), n_classes=8)

    # Compilation du modèle
    unet_model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=total_loss,
        metrics=[IOUScore(threshold=0.5), FScore(threshold=0.5), dice_coeff]
    )

    # Entraînement
    history = unet_model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=10,
        verbose=1
    )

    # Récupération des scores finaux
    final_loss = history.history["val_loss"][-1]
    final_iou = history.history["val_iou_score"][-1]
    final_f1 = history.history["val_f1-score"][-1]
    final_dice = history.history["val_dice_coeff"][-1]

    # Enregistrement des résultats
    results.append([loss_name, final_loss, final_iou, final_f1, final_dice, time.time() - start_time])

# Création du tableau récapitulatif
df_results = pd.DataFrame(results, columns=["Loss Function", "Validation Loss", "IoU Score", "F1-Score", "Dice Coefficient", "Training Time (s)"])

# Affichage du tableau
from IPython.display import display
display(df_results)

# Sauvegarde des résultats sous forme de CSV
df_results.to_csv("/content/drive/My Drive/projet 8/loss_function_comparison.csv", index=False)


🔹 Entraînement avec Dice Loss...
Epoch 1/10
185/185 [==============================] - 4996s 27s/step - loss: 0.5770 - iou_score: 0.3618 - f1-score: 0.4592 - dice_coeff: 0.5787 - val_loss: 0.8592 - val_iou_score: 0.0939 - val_f1-score: 0.1362 - val_dice_coeff: 0.3877
Epoch 2/10
185/185 [==============================] - 1873s 10s/step - loss: 0.4079 - iou_score: 0.5161 - f1-score: 0.6233 - dice_coeff: 0.7650 - val_loss: 0.8355 - val_iou_score: 0.1112 - val_f1-score: 0.1603 - val_dice_coeff: 0.4346
Epoch 3/10
185/185 [==============================] - 948s 5s/step - loss: 0.3618 - iou_score: 0.5532 - f1-score: 0.6660 - dice_coeff: 0.7991 - val_loss: 0.5532 - val_iou_score: 0.3432 - val_f1-score: 0.4551 - val_dice_coeff: 0.6582
Epoch 4/10
185/185 [==============================] - 648s 4s/step - loss: 0.3369 - iou_score: 0.5697 - f1-score: 0.6843 - dice_coeff: 0.8140 - val_loss: 0.3457 - val_iou_score: 0.5518 - val_f1-score: 0.6710 - val_dice_coeff: 0.8017
Epoch 5/10
185/185 [==========

,Loss Function,Validation Loss,IoU Score,F1-Score,Dice Coefficient,Training Time (s)
0,Dice Loss,0.329876,0.549028,0.676359,0.773327,11412.677246
1,Total Loss,1.171774,0.600455,0.716968,0.838799,4696.565830
2,IoU Loss,0.268082,0.533995,0.618252,0.844572,4667.411883
3,Tversky Loss,0.209705,0.500018,0.592262,0.790295,4672.325828
4,Focal Tversky Loss,0.069539,0.525667,0.612751,0.832158,4668.709084


## <font color='red'>5. Conclusion</font><a class="anchor" id="partie5"></a>

Après avoir comparé 5 fonctions de perte (Dice Loss, Total Loss, IoU Loss, Tversky Loss et Focal Tversky Loss),
la Total Loss s'est distinguée comme la fonction de perte la plus performante.

Points clés :
- Meilleur IoU Score (0.600), indiquant une segmentation précise et équilibrée.
- Meilleur Dice Coefficient (0.839), soulignant une bonne similarité entre les masques prédits et réels.
- Temps d'entraînement raisonnable (4697 secondes), malgré une meilleure convergence.

Bien que la Dice Loss ait produit des résultats solides, notamment en Dice Coefficient (0.773), elle a été surpassée par la Total Loss, qui équilibre mieux précision et robustesse sur les classes sous-représentées.

Ces résultats justifient l'utilisation de la Total Loss comme fonction de perte optimale pour les expérimentations suivantes.